# Исследование надежности заемщиков


Импортируем библиотеку pandas. Считаем данные из csv-файла в датафрейм и сохраните в переменную `data`**

In [42]:
import pandas as pd

data = pd.read_csv('data')

**Выведем первые 20 строчек датафрейма `data` на экран.**

In [43]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Выведем основную информацию о датафрейме с помощью метода `info()`.**

In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**Выведем количество пропущенных значений для каждого столбца.**

In [45]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**В двух столбцах есть пропущенные значения. Один из них — `days_employed`.Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [46]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
        data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`.**

In [47]:
data['days_employed'] = data['days_employed'].abs()

**Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.**

In [48]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

**Выведите перечень уникальных значений столбца `children`.**

In [49]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

**В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [50]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [51]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

**Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [52]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
        data.loc[(data['income_type'] == t), 'days_employed'].median()

**Убедимся, что все пропуски заполнены. Проверим и ещё раз выведите количество пропущенных значений для каждого столбца**

In [53]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**Заменим вещественный тип данных в столбце `total_income` на целочисленный.**

In [54]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру. Проверим остальные столбцы.**

In [55]:
data['education'] = data['education'].str.lower()

**Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.**

In [56]:
data.duplicated().sum()

np.int64(71)

In [57]:
data = data.drop_duplicates()

### Категоризация данных

**На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используем собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [58]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [59]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Выведим на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [60]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используем собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучим данные в столбце `purpose` и определим, какие подстроки помогут вам правильно определить категорию.**

In [61]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [62]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуем данные и ответьте на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

Проверим зависимость дохода от наличия детей

In [63]:
data.groupby('children')['total_income'].median()

children
0    142594.0
1    145531.0
2    142594.0
3    145220.0
4    150447.0
5    168460.0
Name: total_income, dtype: float64

In [64]:
data.groupby('children')['total_income'].agg(['median', 'count'])

,median,count
children,,
0,142594.0,14091
1,145531.0,4808
2,142594.0,2052
3,145220.0,330
4,150447.0,41
5,168460.0,9


Количество детей не сильно влияет на доход.
Интересным наблюдением является, что медианная зарплата у людей с 5-ю детьми заметно отличается от остальных.

In [65]:
data_children_grouped = data.groupby('children')['total_income'].median()
data_children_grouped[:3]

children
0    142594.0
1    145531.0
2    142594.0
Name: total_income, dtype: float64

Проверим взаимосвязь между количеством детей и наличием задолженности по кредиту

Пока что мы видим взаимосвязь: чем больше детей, тем меньше вероятность, что у человека с детьми будет задолженность.
Однако пока делать окночательные выводы рано, нужно проверить, какой процент людей с детьми имеел задолженность по кредиту

In [66]:
# посмотрим, сколько из общего количества людей, имевших задолженность, имеют детей
children_pivot_table = pd.pivot_table(data, index=['children'], values=['debt'], aggfunc=['sum', 'count', 'mean'])

# добавим столбец соотношения людей с детьми к общему количеству людей
children_pivot_table.sort_values(by=('mean', 'debt'), ascending=True)


,sum,count,mean
,debt,debt,debt
children,,,
5,0,9,0.000000
0,1063,14091,0.075438
3,27,330,0.081818
1,444,4808,0.092346
2,194,2052,0.094542
4,4,41,0.097561


In [67]:
#Удаляем нерепрезентативные данные
children_pivot_table.drop([5, 3, 4])

,sum,count,mean
,debt,debt,debt
children,,,
0,1063,14091,0.075438
1,444,4808,0.092346
2,194,2052,0.094542


In [68]:
children_pivot_table.loc[1:, ('mean', 'debt')].median()

np.float64(0.09234608985024959)

Медианный процент людей, имевший задолженность:
- `7.5%` — группа людей без детей
- `9.2%` — группа людей с детьми

Интересным наблюдением ялвяется то, что люди с 5-ю детьми совершенно не просрачивают кредит, однако количество значений слишком мало, чтобы делать подобные выводы насчет данной группы людей

##### 3.1 Вывод 

**Вывод:** Чем больше у человека детей, тем меньше вероятность, что он вернет кредит в срок. Исключением являются группы людей с  `3`, `4`, `5`, людьми, ибо количество данных недостаточно, чтобы делать выводы

*Прошлый вывод:* Люди, у которых нет детей, с большей вероятностью верунт кредит в срок.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

Создадим сводную таблицу с семейным статусом и наличием задолженности по кредиту

In [69]:
family_status_pivot_table = pd.pivot_table(data,
                                           index=['family_status_id', 'family_status'],
                                           values=['debt'],
                                           aggfunc=['sum', 'count', 'mean'])

# добавим столбец соотношения людей с долгом к общему количеству людей в одной категории
family_status_pivot_table.sort_values(by=('mean', 'debt'), ascending=True)

,,sum,count,mean
,,debt,debt,debt
family_status_id,family_status,,,
2,вдовец / вдова,63,951,0.066246
3,в разводе,84,1189,0.070648
0,женат / замужем,927,12261,0.075606
1,гражданский брак,385,4134,0.093130
4,Не женат / не замужем,273,2796,0.097639


Глобально можно разделить людей на 2 группы:
- `женат / замужем`
- `не женат / не замужем`

Причем необходимо подметить, что в глобальном ппонимании вторая группа включает в себя следующие категории: `гражданский брак`, `вдовец / вдова`, `в разводе`, `Не женат / не замужем`

Проверим `медианный` и `средний` процент людей, имеющих задолженность в каждой глобальной группе

In [70]:
family_status_pivot_table.loc[1:, ('mean', 'debt')].median()

np.float64(0.08188887166385302)

In [71]:
family_status_pivot_table.loc[1:, ('mean', 'debt')].mean()

np.float64(0.0819158212721453)

Значения отличаются на сотую процента, следовательно можем взять любое. Если округлить до десятой, то получим одинаковый процент.

Процент людей, имевший задолженность:
- `7.6%` для людей из категории `'женат / замужем'`
- `8.2%` для людей из категории `'не женат / не замужем'`


##### 3.2 Вывод

**Вывод:** 
- С большей вероятностью кредит в срок вернут люди, которые ранее, или в данный момент находятся в браке. То есть люди из следующих категорий: `вдовец / вдова`, `в разводе`, `женат / замужем`
- C меньшей вероятностью, кредит в срок вернут люди, которые не находятся в браке. То есть люди из следующих категорий: `гражданский брак`, `Не женат / не замужем`

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Ранее мы создавали функцию, чтобы разбить уровень дохода на категории

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

Создадим сводную таблицу с категорией дохода и наличием задолженности

In [72]:
total_income_pivot_table = pd.pivot_table(data,
                                          index=['total_income_category'],
                                          values=['debt'],
                                          aggfunc=['sum', 'count', 'mean'])

# добавим столбец соотношения людей с долгом к общему количеству людей в одной категории
total_income_pivot_table.sort_values(by=('mean', 'debt'), ascending=True)

,sum,count,mean
,debt,debt,debt
total_income_category,,,
D,21,349,0.060172
B,354,5014,0.070602
A,2,25,0.080000
C,1353,15921,0.084982
E,2,22,0.090909


In [73]:
#Удаляем нерепрезентативные показатели
total_income_pivot_table.drop(['D', 'A', 'E'])

,sum,count,mean
,debt,debt,debt
total_income_category,,,
B,354,5014,0.070602
C,1353,15921,0.084982


##### 3.3 Вывод

**Вывод:** Чем выше доход человека, тем выше вероятность, что он вернет кредит в срок.
- С наибольшей вероятностью кредит в срок вернут люди из категории `'B'`
- С наименьшей вероятностью кредит в срок вернут люди из категории `'С'`, 

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

Создадим сводную таблицу с целью кредита и наличием задолженности

In [74]:
purpose_category_pivot_table = pd.pivot_table(data,
                                              index=['purpose_category'],
                                              values=['debt'],
                                              aggfunc=['sum', 'count', 'mean'])

# добавим столбец соотношения людей с долгом к общему количеству людей в одной категории
purpose_category_pivot_table.sort_values(by=('mean', 'debt'), ascending=True)

,sum,count,mean
,debt,debt,debt
purpose_category,,,
операции с недвижимостью,780,10751,0.072551
проведение свадьбы,183,2313,0.079118
получение образования,369,3988,0.092528
операции с автомобилем,400,4279,0.093480


Большой процент людей просрачивает возврат кредита по категории `получение образования`. Рождается гипотеза, что кредит берут студенты, и так как у них нет постоянного источника дохода, то они не могут вовремя погасить долг. Проверим данную гипотезу

In [75]:
data.loc[data['purpose_category'] == 'получение образования', 'income_type'].value_counts()

income_type
сотрудник      2066
компаньон       947
пенсионер       719
госслужащий     256
Name: count, dtype: int64

In [76]:
# проверим, есть ли в исходном датафрейме строки, удовлетворяющие условию
data[(data['purpose_category'] == 'получение образования') & (data['income_type'] == 'студент')]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category


Таблица пуста, следовательно среди взявших кредит на получение образования действительно нет ни одного студента, следовательно мы можем отмести данную гипотезу

##### 3.4 Вывод

**Вывод:** 
- С наибольшей вероятностью кредит в срок вернут люди из категории `'операции с недвижимостью'`, `'проведение свадьбы'`
- С наименьшей вероятностью кредит в срок вернут люди из категории `'операции с автомобилем'`, `'получение образования'`

*Интересное наблюдениие: среди взявших кредит на образование нет ни одного студента*

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* Так как у нас одинаковое количество пропусков в столбцах `'days_employed'` и `'total_income'`, то существует вероятность, что причиной пропусков может быть:
- человек не трудоустраивался официально
- человек не декларировал свой доход
- человек может пользоваться разными банками: один зарплатный, который хранит информацию о зарплате, а другой — тот, в котором он хочет взять кредит, может иметь более выгодную процентную ставку
- если не указан доход, то общий трудовой стаж в днях может автоматически не указываться
- если не указан трудовой стаж, то может автоматически не указываться доход
- ошибки в передаче данных
- если данная таблица составлялась из другой таблицы, в которой были ошибки

In [77]:
# Посмотрим, какие категории граждан подвержены пропуску данных в days_employed
# Так как выше мы уже обрабатывали таблицу, избавляясь от пропусков, то используем костыли, для быстрой проверки
temp_data = pd.read_csv('data')

temp_data = temp_data[['income_type', 'days_employed']].copy()
temp_data = temp_data[temp_data['days_employed'].isna()]
temp_data.drop_duplicates(inplace=True)
temp_data

,income_type,days_employed
12,пенсионер,NaN
26,госслужащий,NaN
65,компаньон,NaN
82,сотрудник,NaN
5936,предприниматель,NaN


Среди категорий есть `госслужащий`, так что гипотеза, что человек не трудоустраивался официально — выглядит неправдоподобно

#### 3.6 Объясним, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

In [78]:
print(data['total_income'].min())
print(data['total_income'].max())

print(data['total_income'].mean())
print(data['total_income'].median())

data['total_income'].mean() - data['total_income'].median()

20667
2265604
165342.74576906848
142594.0


np.float64(22748.745769068482)

- `220667` — Минимальное значение
- `2265604` — Максимальное значение
- `165342` — Среднее значение
- `142594` — Медианное значение

Разница между медианным и средним значением составляет `22748`

*Ответ:* При использовании медианного значения, мы "избегаем" учет аномально больших значений, которые влияют на число. К примеру, в списке `[100, 100, 100, 100, 10000]` среднее значение будет соствлять `2080`. В то время, как медианное значение будет составлять `100`. Следовательно, если мы хотим заменить пропуски значениями, приближенными к среднему значению большинства (грубо говоря, "исключая" аномально большие значения), то мы заполняем их медианным значением

### Шаг 4: общий вывод.

**С наибольшей вероятностью кредит в срок вернут следующие категории граждан:**
- `'0 детей'`
- `Были или находятся в браке`: `'женат / замужем'`, `в разводе`, `'вдова / вдовец'` 
- `'B'` — люди с доходом от 200001 до 1000000
- `'операции с недвижимостью'`, `'проведение свадьбы'`

**C Наименьшей же вероятность кредит в срок вернут следующие категории граждан:**
- `'есть дети'`
- `'не женат / не замужем'`, `'гражданский брак'`
- `'С'` — люди с доходом от 50001 до 200000
- `'операции с автомобилем'`, `'получение образования'`

Чем меньше детей и выше доход, тем больше вероятность, что кредит будет возвращен в срок. Можно добавить, что люди, не состоящие в браке, имеющие детей, а так же с низким доходом являют собой ненадежных заемщиков. В целом, среди всех вышеописанных данных нельзя выделить особенно "хорошие" или "плохие" категории людей, которые брали кредит. Количество людей, которые не вернули кредит вовремя варьируется от `6%` до `10%`.

<br>
<br>

Для получения выводов мы рассматривали такие данные, как: `тип занятости`, `трудовой стаж`, `уровень дохода`, `количество детей`,  `цель кредита`. Во время работы были не приняты во внимание такие нерепрезентативные данные, как `уровень дохода: E, D, А`, `количество детей: 3, 4, 5`, так как людей в данных группах было слишком мало, чтобы сделать выводы

**Проделанная работа:**
- Пропущенные значения из столбца `total_income` были заполнены медианным значением
- Пропущенные значения из столбца `days_employed` были заполнены медианным значением по типу занятости `income_type`
<br><br>
- Аномальные значения из столбца столбца `days_employed` являли собой отрицательные числа. Исправлены на положительные значения
- Аномальные значений столбца `children`: `-1`, `20`. Исправлены, путем удаления строк, содержащих данные значения
<br><br>
- В столбце `education` все значения были приведены к единому стилю, тем самым мы избавились от неявных дубликатов в данном столбце
<br><br>
- Уровень дохода был разделен на 5 категорий: `A`, `B`, `C`, `D`, `E`
- Цели кредита были разделены на 4 категории: `'операции с автомобилем'`, `'операции с недвижимостью'`, `'проведение свадьбы'`, `'получение образования'`